## Homework 1

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.


## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

In [13]:
import pandas as pd

df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

In [14]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [15]:
df.shape

(3066766, 19)

Read the data for January. How many columns are there?

* 16
* 17
* 18
* **19**

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

In [16]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [17]:
df.duration.std()

np.float64(42.59435124195458)

What's the standard deviation of the trips duration in January?

* 32.59
* **42.59**
* 52.59
* 62.59

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

In [18]:
df_update = df[(df.duration >= 1) & (df.duration <= 60)]

In [19]:
len(df_update)/len(df)

0.9812202822125979

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* **98%**

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

In [20]:
features = ['PULocationID', 'DOLocationID']

df_update[features] = df_update[features].astype(str)

C:\Users\marti.MARTIN\AppData\Local\Temp\ipykernel_16048\2996617965.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_update[features] = df_update[features].astype(str)


In [22]:
from sklearn.feature_extraction import DictVectorizer

vect_dict = df_update[features].to_dict(orient='records')

dv = DictVectorizer()
X = dv.fit_transform(vect_dict)

In [23]:
X.shape

(3009173, 515)

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* **515**
* 715

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters, where duration is the response variable
* Calculate the RMSE of the model on the training data

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [26]:
target = 'duration'
y = df_update[target].values

lr = LinearRegression()
lr.fit(X, y)

y_pred = lr.predict(X)

root_mean_squared_error(y, y_pred)

7.649261931816197

What's the RMSE on train?

* 3.64
* **7.64**
* 11.64
* 16.64

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

In [27]:
df_val = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')

In [28]:
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)

In [29]:
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

In [30]:
df_val[features] = df_val[features].astype(str)

In [31]:
val_dict = df_val[features].to_dict(orient='records')
X_val = dv.transform(val_dict)
y_val = df_val[target].values

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.8118186871593

What's the RMSE on validation?

* 3.81
* **7.81**
* 11.81
* 16.81